In [1]:
import pandas as pd
from utils import fillna__
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.options.io.excel.xlsx.writer = 'xlsxwriter'

# 财务绩效处理
fin = r"C:\onedrivers\OneDrive - jsnu.edu.cn\projects\digital\03.企业绩效数据\datas\全国上市企业财务绩效汇总表.csv"
fin = pd.read_csv(fin, index_col=None, header=0, dtype={'股票代码':'object'})  # , parse_dates=['上市日期']
fin['营业费用增长率'] = (fin['销售费用增长率'] + fin['管理费用增长率'])/2
# 净资产收益率(+)/投入资本回报率(+)/成本费用利润率(+)/资产报酬率(+)
# 总资产周转率(+)/应付账款周转率(-)/应收账款周转率(+)/存货周转率(-)
# 资产负债率(-)/速动比率(+)/权益乘数(+)/现金资产比率(+)/营运指数(+)
# 净利润增长率(+)/固定资产增长率(+)/营业收入增长率(+)/营业费用增长率(-)
# 财务杠杆(+)/经营杠杆(+)
fin_colums=['股票代码', '截止年份', '净资产收益率（ROE）B', '投入资本回报率（ROIC）', '成本费用利润率', '资产报酬率B',' 总资产周转率B', '应付账款周转率B', '应收账款周转率B', '存货周转率B', '资产负债率', '速动比率', '权益乘数', '现金资产比率', '营运指数', '净利润增长率B', '固定资产增长率B', '营业总收入增长率', '营业费用增长率', '财务杠杆', '经营杠杆']
unfin = r"C:\onedrivers\OneDrive - jsnu.edu.cn\projects\digital\03.企业绩效数据\datas\全国上市企业非财务绩效汇总表.csv"
unfin = pd.read_csv(unfin, index_col=None, header=0, dtype={'股票代码':'object'})  # , parse_dates=['上市日期']
# 发明专利申请数(+)：loss
# 研发人员占比(+)/研发营收比(+)/
# 提供岗位增长率(+)/员工收入增长率(+)/
# 股权集中度(+)/两权分离度(+)/董事会规模(+)/独董比例(+)
# 供应商集中程度(+)/客户集中程度(+)
# 缺失值太多：'纳税总额A','捐赠总额',————>支付税费增长率(+)/
unfin_colums=['股票代码','截止年份','研发人员数量占比(%)','研发营收比(%)','员工人数','职工薪酬','主营业务收入','股权集中度4(%)','两权分离度(%)','董事会规模','独董比例','本期采购额占年度采购总额比例(%)','本期销售额占年度销售总额比例(%)']
fin = pd.merge(unfin[unfin_colums],fin[fin_colums],on=['股票代码','截止年份'],how='outer')
fin = fillna__(fin,'股票代码',rate=0.25)

# 处理部分非财务指标
fin['人力资源效率'] = fin['主营业务收入'] / fin['员工人数']
fin['董事会规模'] = fin['董事会规模'].apply(lambda x: np.log(x+1))
fin['提供岗位增长率'] = fin.groupby('股票代码')['员工人数'].pct_change(1, fill_method='ffill')
fin['员工收入增长率'] =fin.groupby('股票代码')['职工薪酬'].pct_change(1, fill_method='ffill')
fin = fin[['股票代码','截止年份','净资产收益率（ROE）B','投入资本回报率（ROIC）','成本费用利润率','资产报酬率B','总资产周转率B','应付账款周转率B','应收账款周转率B','存货周转率B','资产负债率','速动比率','权益乘数','现金资产比率','营运指数','净利润增长率B','固定资产增长率B','营业总收入增长率','营业费用增长率','财务杠杆','经营杠杆','研发人员数量占比(%)','研发营收比(%)','人力资源效率','提供岗位增长率','员工收入增长率','股权集中度4(%)','两权分离度(%)','董事会规模','独董比例','本期采购额占年度采购总额比例(%)','本期销售额占年度销售总额比例(%)']]

In [2]:
# 企业信息数据
info =  pd.read_csv(r'C:/datas/企业绩效数据/其他数据/STK_LISTEDCOINFOANL.txt',sep='\t', header=0, dtype={'Symbol': 'object'}, parse_dates=['EndDate','LISTINGDATE'])
info = info[(info.EndDate.dt.month == 12)][['Symbol','EndDate','LISTINGDATE','LISTINGSTATE','IndustryCodeC']]
info.rename(columns={'Symbol':'股票代码','EndDate':'截止年份','LISTINGSTATE':'上市状态','IndustryCodeC':'行业代码','LISTINGDATE':'上市日期'},inplace=True)
info['截止年份'] = info['截止年份'].dt.year.astype(int)
info['上市日期'] = info['上市日期'].dt.year.astype(int)
info['上市状态'] = info['上市状态'].apply(lambda x: 1 if x=='正常上市' else 0)
grant = pd.read_excel(r'C:/datas/企业绩效数据/非财务绩效数据/专利授权情况表(1992-2017)142534725(仅供盐城工学院使用)/上市公司专利申请和授权数据.xlsx', dtype={'股票代码':'object'})
# grant['当年独立申请专利总和'] = grant['当年独立申请专利总和'].apply(lambda x : np.log(1+x))
# grant['当年独立获得专利总和'] = grant['当年独立获得专利总和'].apply(lambda x : np.log(1+x))
# grant['申请专利数'] = (grant['当年独立申请专利总和']+grant['当年独立获得专利总和'])/2
grant['申请专利数'] = grant['当年独立申请专利总和'].apply(lambda x : np.log(1+x))
grant = grant[(grant['公司类型']=='集团公司合计')][['股票代码','year','申请专利数']]
grant = fillna__(grant,'股票代码', rate=0.25)
grant.rename(columns={'year':'截止年份'}, inplace=True)
# 数字技术应用(+)/商业模式变革(+)/智能制造(+)/现代信息系统(+)
digital= pd.read_csv(r'C:\onedrivers\OneDrive - jsnu.edu.cn\projects\digital\02.数字化转型水平测度与评价\datas\数字化转型程度2022(未筛选).csv',dtype={'股票代码': 'object'})
for name in ['数字技术应用z','互联网商业模式z','智能制造z','现代信息系统z']:
    digital[name] = digital[name].apply(lambda x : np.log(1+x))
digital_colums=['股票代码','截止日期','wufei','yuanchun','csmar','lishouxi','数字技术应用z','互联网商业模式z','智能制造z','现代信息系统z']
digital=digital[digital_colums].sort_values(['股票代码','截止日期'])
digital.rename(columns={'截止日期':'截止年份','数字技术应用z':'数字技术应用','互联网商业模式z':'互联网商业模式','智能制造z':'智能制造','现代信息系统z':'现代信息系统'}, inplace=True)
info = pd.merge(info,grant,on=['股票代码','截止年份'],how='outer')
info = pd.merge(info,digital,on=['股票代码','截止年份'],how='outer')
info = info[(info['截止年份']>2006)&(info['截止年份']<2023)]

In [5]:
dataset = pd.merge(fin,info,on=['股票代码','截止年份'],how='outer')
get_na_codes = lambda dataset, name : set(dataset[dataset[name].isna()]['股票代码'].to_list())
sts_ids = set(dataset[dataset['上市状态']==0]['股票代码'].tolist())
dataset.dropna(subset=['行业代码'],inplace=True)
#  顺便剔除了这些数据 ：'上市状态', '行业代码','上市日期', '纳税总额A', '捐赠总额',,'人力资源效率',
needs = [
    '股票代码', '截止年份', '净资产收益率（ROE）B', '投入资本回报率（ROIC）', '成本费用利润率', '资产报酬率B', '总资产周转率B', '应付账款周转率B', '应收账款周转率B', '存货周转率B', '资产负债率', '速动比率', '权益乘数', '现金资产比率', '营运指数', '净利润增长率B', '固定资产增长率B', '营业总收入增长率', '营业费用增长率', '财务杠杆', '经营杠杆', 
    '研发人员数量占比(%)', '研发营收比(%)', '提供岗位增长率', '员工收入增长率',
    '股权集中度4(%)', '两权分离度(%)', '董事会规模', '独董比例', 
    '本期采购额占年度采购总额比例(%)', '本期销售额占年度销售总额比例(%)','申请专利数',
     '数字技术应用', '互联网商业模式', '智能制造', '现代信息系统', 'wufei', 'yuanchun', 'csmar','lishouxi'
]
dataset = dataset[(dataset['行业代码'].str.startswith('C'))]
dataset = dataset[~(dataset['行业代码'].str.endswith('39'))]
dataset = dataset[~dataset['股票代码'].isin(sts_ids)]
dataset = dataset[(dataset['截止年份']>2012)&(dataset['上市日期']<2021)]
dataset = dataset[~dataset['股票代码'].isin(get_na_codes(dataset,'lishouxi'))]
dataset = fillna__(dataset[needs],'股票代码',rate=0.2).sort_values(by=['股票代码','截止年份'])
dataset

,股票代码,截止年份,净资产收益率（ROE）B,投入资本回报率（ROIC）,成本费用利润率,资产报酬率B,总资产周转率B,应付账款周转率B,应收账款周转率B,存货周转率B,...,本期销售额占年度销售总额比例(%),申请专利数,数字技术应用,互联网商业模式,智能制造,现代信息系统,wufei,yuanchun,csmar,lishouxi
39,000004,2013,0.039627,0.029738,0.183205,0.047724,0.334591,2.310006,12.243145,0.527699,...,37.96,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.379427,0.280618,0.000000
40,000004,2014,0.103513,0.079137,0.270178,0.068684,0.277753,1.582232,24.791980,0.357804,...,44.02,1.386294,0.000000,0.000000,0.000000,0.000000,0.000000,0.355872,0.255612,0.000000
41,000004,2015,0.042766,0.032411,0.097357,0.028676,0.326897,1.180446,53.911663,0.351408,...,20.15,2.564949,0.000000,0.000000,0.000000,0.000000,0.000000,0.239521,0.268059,0.000000
42,000004,2016,0.232602,0.226309,0.175808,0.156791,0.924405,4.476773,167.604472,1.466653,...,8.98,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.354767,0.244403,0.000000
43,000004,2017,0.032528,0.023170,0.067536,0.034208,0.562797,10.561173,24.833489,4.855790,...,18.30,0.000000,0.000000,1.098612,0.000000,0.000000,1.609438,0.774878,0.295534,0.298641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59231,688686,2021,0.126012,0.112538,0.573878,0.124256,0.341163,4.451873,2.370634,2.592239,...,35.34,4.941642,1.386294,0.000000,3.091042,0.000000,3.496508,4.712105,0.410381,0.816232
59232,688686,2022,0.122735,0.109802,0.446093,0.119535,0.403168,4.963620,2.320462,2.574553,...,46.09,4.663439,0.693147,1.098612,3.135494,1.098612,3.610918,5.452663,0.412088,0.980619
59267,688698,2020,0.170257,0.111715,0.200996,0.132108,0.749514,3.805550,4.563628,3.485082,...,14.92,3.044522,1.386294,0.000000,1.945910,1.791759,0.000000,2.324987,0.365297,0.000078
59268,688698,2021,0.155222,0.128555,0.195168,0.111549,0.698676,4.389992,4.622608,3.066133,...,18.53,3.688879,1.945910,0.000000,3.583519,1.098612,1.386294,3.043203,0.389688,0.221424


In [6]:
import mapclassify as mc

# 确定研究对象:对数据进行分层找出大约20%的数字化转型有效企业
# 观察各种数字化转型指数的聚类情况，思考如何定义数字化企业比较好？--- 可供考虑的：wufei/yuanchun/csmar/lishouxi
print(f'保留大约{int(dataset.shape[0]*0.20)}的企业')
# digital = dataset[['股票代码', '截止年份', 'wufei', 'yuanchun', 'csmar','lishouxi']]
# print(mc.NaturalBreaks(digital['lishouxi'], 2))
# wufei>2/yuanchun>2.53/lishouxi>0.46
data = dataset.copy()
data = data[(data['yuanchun']>2)&(data['lishouxi']>0.46)].drop([ 'wufei', 'yuanchun', 'csmar','lishouxi'],axis=1).dropna()
data.to_csv('./datas/dataset.csv',index=False,encoding='utf-8')
data

保留大约3528的企业


,股票代码,截止年份,净资产收益率（ROE）B,投入资本回报率（ROIC）,成本费用利润率,资产报酬率B,总资产周转率B,应付账款周转率B,应收账款周转率B,存货周转率B,...,两权分离度(%),董事会规模,独董比例,本期采购额占年度采购总额比例(%),本期销售额占年度销售总额比例(%),申请专利数,数字技术应用,互联网商业模式,智能制造,现代信息系统
45,000004,2019,-0.004540,-0.003744,0.045469,0.004754,0.117298,1.339469,0.742593,4.651037,...,22.7330,9.219696,0.428571,91.83,15.54,4.234107,2.302585,2.484907,1.098612,1.098612
108,000008,2018,0.046860,0.045169,0.196867,0.046609,0.242605,2.016633,0.919671,1.881115,...,0.0000,11.329387,0.416667,13.83,43.89,3.332205,3.218876,1.386294,3.663562,2.484907
109,000008,2019,0.061065,0.054076,0.197703,0.056981,0.286096,1.975366,0.960878,2.137038,...,0.0000,11.443490,0.333333,17.78,26.80,4.219508,3.258097,0.000000,3.555348,1.945910
417,000030,2020,0.115520,0.082410,0.085494,0.067531,0.811730,3.864573,6.676066,9.774570,...,2.7348,10.248636,0.272727,16.42,73.51,4.060443,2.079442,0.000000,2.397895,0.000000
418,000030,2021,0.092734,0.068497,0.066624,0.057768,0.874976,4.356312,7.482725,10.435997,...,2.7546,10.302700,0.300000,12.46,66.48,3.332205,3.044522,0.693147,2.995732,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58977,688599,2021,0.109258,0.043780,0.053583,0.046967,0.815164,6.002502,7.173230,3.843288,...,1.0059,10.394274,0.333333,32.21,9.05,5.513429,3.044522,0.000000,3.091042,0.693147
59180,688676,2022,0.106166,0.055751,0.059789,0.040424,0.738688,3.072949,3.208040,2.396251,...,0.4343,8.785081,0.333333,32.38,23.54,3.135494,4.644391,2.708050,4.025352,2.397895
59192,688678,2022,0.115674,0.080486,0.202634,0.068679,0.448270,2.404666,2.705787,2.984502,...,0.0000,8.856376,0.600000,16.22,46.14,3.688879,2.197225,0.000000,3.295837,1.386294
59231,688686,2021,0.126012,0.112538,0.573878,0.124256,0.341163,4.451873,2.370634,2.592239,...,0.8451,8.432506,0.600000,30.24,35.34,4.941642,1.386294,0.000000,3.091042,0.000000
